In [70]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import LSTM,Dense,Input,Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [33]:
data = pd.read_csv('Data.csv')
data.head()

,INPUT,RESPONSE,Unnamed: 2
0,Hi there!,Hello!,NaN
1,How are you today?,"I'm doing well, thank you. How about yourself?",NaN
2,"I'm good too, thanks. Just a bit tired.",I understand. Long day?,NaN
3,"Yeah, it was. Lots of meetings.","Oh, I hate those.",NaN
4,"Me too. So, what are you up to today?",Just relaxing. Any plans for you?,NaN


In [48]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10102 entries, 0 to 10101
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   INPUT      10102 non-null  object
 1    RESPONSE  10095 non-null  object
dtypes: object(2)
memory usage: 158.0+ KB


In [34]:
data = data.drop('Unnamed: 2',axis=1)
data.head()

,INPUT,RESPONSE
0,Hi there!,Hello!
1,How are you today?,"I'm doing well, thank you. How about yourself?"
2,"I'm good too, thanks. Just a bit tired.",I understand. Long day?
3,"Yeah, it was. Lots of meetings.","Oh, I hate those."
4,"Me too. So, what are you up to today?",Just relaxing. Any plans for you?


In [35]:
data_input = data['INPUT']
data_input = [str(item) for item in data_input]
data_input[:4]

['Hi there!',
 'How are you today?',
 "I'm good too, thanks. Just a bit tired.",
 'Yeah, it was. Lots of meetings.']

In [36]:
data_output = data[' RESPONSE']
data_output = [str(item) for item in data_output]
data_output[:4]

['Hello!',
 "I'm doing well, thank you. How about yourself?",
 'I understand. Long day?',
 'Oh, I hate those.']

In [37]:
data_input = data_input[:10000]
data_output = data_output[:10000]

In [47]:
len(data_output)

10000

In [58]:
num_encoder_tokens = len(tokenizer.word_index) + 1
print(f"Number of Encoder Tokens: {num_encoder_tokens}")

Number of Encoder Tokens: 2230


In [39]:
tokenizer = Tokenizer()
input_tokens = tokenizer.fit_on_texts(data_input)
output_tokens = tokenizer.fit_on_texts(data_output)

In [40]:
input_sequences = tokenizer.texts_to_sequences(data_input)
output_sequences = tokenizer.texts_to_sequences(data_output)

In [41]:
input_data = pad_sequences(input_sequences,maxlen=25,padding='post')
output_data = pad_sequences(output_sequences,maxlen=25,padding='post')

In [60]:
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(len(tokenizer.word_index), 128, mask_zero=True)(encoder_inputs)
encoder_lstm = LSTM(512, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

In [63]:
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(len(tokenizer.word_index), 128, mask_zero=True)(decoder_inputs)
decoder_lstm = LSTM(512, return_sequences=True, return_state=True, dropout=0.3, recurrent_dropout=0.3)
decoder_dense = Dense(len(tokenizer.word_index), activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [71]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
optimizer = Adam(learning_rate=0.0005, beta_1=0.9, beta_2=0.999, decay=1e-6)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

C:\Users\Brijesh AH\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\optimizers\base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


In [72]:
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_6       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_8       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, None, 128) │    285,312 │ input_layer_6[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_1         │ (None, None)      │          0 │ input_layer_6[0]… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_3         │ (None, None, 128) │    285,312 │ input_layer_8[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_2 (LSTM)       │ [(None, 512),     │  1,312,768 │ embedding_1[0][0… │
│                     │ (None, 512),      │            │ not_equal_1[0][0] │
│                     │ (None, 512)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_3 (LSTM)       │ [(None, None,     │  1,312,768 │ embedding_3[0][0… │
│                     │ 512), (None,      │            │ lstm_2[0][1],     │
│                     │ 512), (None,      │            │ lstm_2[0][2]      │
│                     │ 512)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_9       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, None,      │  1,143,477 │ lstm_3[0][0]      │
│                     │ 2229)             │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 4,339,637 (16.55 MB)

 Trainable params: 4,339,637 (16.55 MB)

 Non-trainable params: 0 (0.00 B)